In [45]:
import pandas as pd
import numpy as np
import re
from calendar import month_abbr

import unicodedata

# TO-DO

- que se tome en cuenta si los primeros 2 datos son cm (actualmente se asume que son pulgadas)
  - ej: 27x25 cm (10x5 in)
  - NO NECESARIO
- checar tipo y téncnica
- que los precios estén bien
- sacar promedio para el precio estimado
- que el formato de la fecha sea el correcto

# limpieza de datos

In [109]:
def get_sold(df):
    
    sold = df[df['precio_venta'].str.contains('upcoming', case =False) == False]
    sold = sold[sold['precio_venta'].str.contains('live now', case =False) == False]
    sold = sold[sold['precio_venta'].str.contains('results coming soon', case =False) == False]
    sold = sold[sold['precio_venta'].str.contains('not sold', case =False) == False]
    
    return sold

In [183]:
df = pd.read_csv('../datasets/MutualArtLimpio.csv')
df_2 = pd.read_csv('../datasets/data_mutualart_completo.csv')


df_artistas = pd.read_csv('../datasets/lista_artistas.csv', encoding='utf8')

df = get_sold(df)
df_2 = get_sold(df_2)

In [115]:
#df_2.autor.unique() #641 authors

In [116]:
df.columns

Index(['ID', 'precio estimado', 'precio_venta', 'fecha', 'autor',
       'casa subasta', 'tipo', 'tecnica', 'dimensiones',
       'precio estimado dolares', 'precio venta dolares'],
      dtype='object')

# Clean date


In [186]:
def clean_date(df_orig):
    '''
    Cambia el formato a YYYY-MM-DD
    '''
    lower_ma = [m.lower() for m in month_abbr]
    df = df_orig.copy()
    
    months = df['fecha'].str.extract('^([a-z]{3})') 
    months = months[0].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')
    years = df['fecha'].str.extract('([0-9]{4})')
    days = df['fecha'].str.extract('([0-9]{2}),')

    df['fecha'] = years[0]+'-'+months.astype(str)+'-'+days[0]
    df['fecha'] = pd.to_datetime(df_clean['fecha'])
    
    return df

In [189]:
df_clean = clean_date(df)

# Clean nombres artistas

- from the MutualArtLimpio.csv dataset hay 95 artistas diferentes (con solo obras vendidas)
- 

artistas que deberían hacer match
- cesar villacres -> cesar a.villacres
- emiliano di cavalcanti -> emiliano de cavalvanti
- graciela rodo-boulanger -> graciela rodo boulanger
- ignacio iturria -> ignacio de iturria
- jesus-rafael soto -> jesus rafael soto
- joaquin torres garcia -> joaquin torres-garcia
- jose maria mijares -> jose mijares
- leopoldo romanac -> leopoldo romaã‘ach
- ricardo martinez de hoyos -> ricardo martinez

In [193]:
def clean_autor(df):
    '''
    Limpiar artistas de mutual art y los modifica para
    ser iguales a la lista de artistas acordada
    
    Es probable que se tenga que modificar esta función para añadir nombres adicionales.
    Hay artistas que no son de la lista
    
    SUGERENCIA: quitarle los guiones a todos los nombres (de la tabla de referencia)
    '''
    df['autor'] = df['autor'].str.replace('cesar villacres', 'cesar a.villacres')
    df['autor'] = df['autor'].str.replace('emiliano di cavalcanti', 'emiliano de cavalvanti')
    df['autor'] = df['autor'].str.replace('graciela rodo-boulanger', 'graciela rodo boulanger')
    df['autor'] = df['autor'].str.replace('jesus-rafael soto', 'jesus rafael soto')
    df['autor'] = df['autor'].str.replace('joaquin torres garcia', 'joaquin torres-garcia')
    df['autor'] = df['autor'].str.replace('ignacio iturria', 'ignacio de iturria')
    df['autor'] = df['autor'].str.replace('jose maria mijares', 'jose mijares')
    df['autor'] = df['autor'].str.replace('emiliano di cavalcanti', 'emiliano de cavalvanti')
    df['autor'] = df['autor'].str.replace('ricardo martinez de hoyos', 'ricardo martinez')
        
    return df
    

In [194]:
len(df['autor'].unique())

95

In [195]:
df_artistas['autor'] = df_artistas['name']+' '+df_artistas['last_name']
df_artistas

,last_name,name,born,death,notes,autor
0,abela,eduardo,1889,1965.0,NaN,eduardo abela
1,alfonzo,carlos,1950,1991.0,NaN,carlos alfonzo
2,do amaral,tarsila,1886,1971.0,NaN,tarsila do amaral
3,anguiano,raul,1915,2006.0,NaN,raul anguiano
4,atl,dr.,1875,1964.0,NaN,dr. atl
...,...,...,...,...,...,...
110,velasco,jose maria,1840,1912.0,NaN,jose maria velasco
111,velasquez,jose antonio,1906,1985.0,NaN,jose antonio velasquez
112,villacres,cesar a.,1880,1941.0,NaN,cesar a. villacres
113,zarraga,angel,1886,1946.0,NaN,angel zarraga


In [196]:
df = clean_autor(df)

ma_authors = df.groupby(['autor']).count()['ID'].reset_index()
ma_authors.sort_values(by=['ID'], ascending=False)
ma_authors['temp_col'] = 1

In [197]:
merged_df = ma_authors.merge(df_artistas, how='outer', on='autor')
#merged_df[merged_df['autor']]
merged_df.shape

(147, 8)

In [198]:
merged_df[merged_df['temp_col'].isna() == True].sort_values(by=['autor'])

,autor,ID,temp_col,last_name,name,born,death,notes
143,abigail varela,NaN,NaN,varela,abigail,1948.0,NaN,NaN
103,agustin cardenas,NaN,NaN,cardenas,agustin,1927.0,2001.0,ESCULTURAS
120,agustin fernandez,NaN,NaN,fernandez,agustin,1928.0,2006.0,NaN
132,albert lynch,NaN,NaN,lynch,albert,1851.0,1930.0,NaN
110,alejandro colunga,NaN,NaN,colunga,alejandro,1948.0,NaN,NaN
106,alfredo castaã‘eda,NaN,NaN,castaã‘eda,alfredo,1938.0,2010.0,NaN
140,alicia penalba,NaN,NaN,penalba,alicia,1918.0,1982.0,NaN
97,antonio berni,NaN,NaN,berni,antonio,1905.0,1981.0,NaN
135,armando morales,NaN,NaN,morales,armando,1927.0,2011.0,NaN
131,august lohr,NaN,NaN,lohr,august,1843.0,1919.0,NaN


In [199]:
merged_df[merged_df['name'].isna() == True]

#38 artistas de la lista de artistas que no hacen match con datos de mutual art
#58 de mart que no hacen match con la lista de artistas

,autor,ID,temp_col,last_name,name,born,death,notes
8,antonio gonzalez orozco,1.0,1.0,NaN,NaN,NaN,NaN,NaN
11,arthur luiz piza,1.0,1.0,NaN,NaN,NaN,NaN,NaN
18,cesar a.villacres,41.0,1.0,NaN,NaN,NaN,NaN,NaN
19,cesar izquierdo,1.0,1.0,NaN,NaN,NaN,NaN,NaN
20,cristina cruz,1.0,1.0,NaN,NaN,NaN,NaN,NaN
21,daniel evo,1.0,1.0,NaN,NaN,NaN,NaN,NaN
24,debbie reda,1.0,1.0,NaN,NaN,NaN,NaN,NaN
30,emiliano de cavalvanti,58.0,1.0,NaN,NaN,NaN,NaN,NaN
31,emilio behar,1.0,1.0,NaN,NaN,NaN,NaN,NaN
38,francis luis mora,1.0,1.0,NaN,NaN,NaN,NaN,NaN


# Clean dimensiones
(height va primero)

idea 1:
- detectar si está en inches o cm (eliminar los que no incluyen medidas)
- extraer fracciones
- extraer los primeros 2 números


In [4]:

def fraction_char_replace(s):
    '''
    Reemplaza las fracciones representadas como un solo caracter y las convierte en decimales
    ej: ¾, ½ se convierten en .75 y .5
    Args: 
        s(str)- string de las dimensiones de una obra
    '''
    
    for c in s:
        try:
            name = unicodedata.name(c)
        except ValueError:
            continue
        if name.startswith('VULGAR FRACTION'):
            normalized = unicodedata.normalize('NFKC', c)
            numerator, _slash, denominator = normalized.partition('⁄')
            
            frac = int(numerator)/int(denominator)
            s = s.replace(c, str(frac))
            #yield c, numerator, denominator
            
            
    return s
            
#TO-DO
#- encontrar source de la fuente

In [5]:
def fraction_str_replace(s):
    '''
    Se asume que dentro de cada registro, se encuentra un 
    Formas de detectat una fraccion de la forma a/b:
    
    num a/b
    
    aaa/b
    
    num-a/b
    
    num.a/b
    
    ojo: existe .groups()
    '''
    if(re.search(' [0-9]?[0-9]/[0-9]?[0-9]', s)):  #tipo num a/b
        
        #es un ciclo porque se debe de tratar cada fraccion en todo el string
        for m in re.findall('([0-9]?[0-9])/([0-9]?[0-9])',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0','',str(frac))
            
            pattern = ' '+str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    elif(re.search('[0-9][0-9]{2}?/[0-9][0-9]?',s)): #tipo aaa/b
        
        for m in re.findall('([0-9][0-9][0-9])/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            pattern = str(num)+'/'+str(denom)
            
            s = re.sub(pattern, str(frac),s)
        
    elif(re.search('[0-9][0-9]?-[0-9][0-9]?/[0-9]?[0-9]',s)):
        #tipo num-a/b
        
        for m in re.findall('([0-9][0-9]?[0-9]?)/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0','',str(frac))
            pattern = '-'+str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    elif(re.search('[0-9]\.[0-9][0-9]?/[0-9]?[0-9]',s)):
        #tipo num.a/b
        
        for m in re.findall('([0-9][0-9]?[0-9]?)/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0\.','',str(frac))
            pattern = str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    else: 
        resp = 'no se detectó'
    
    return s

#TO-DO:
#simplificar código
#talvez se pueden hacer menos ifs
#mejorar comentarios

In [6]:
#test de la funcion anterior

#fracciones = df[df['dimensiones'].str.contains('[0-9]\/[0-9]') == True].copy()#.head(5)
#fracciones['tipo_frac'] = fracciones.dimensiones.apply(lambda x: fraction_str_replace(x))

In [7]:
#fracciones
#ejem = fracciones.iloc[2]['dimensiones']

In [8]:
def is_inches(s):
    '''
    
    '''
    no_measures_ids = ['various sizes', 'dimensions variable', 'property from the estate of bernard chappard\xa0 ...']
    inches_ids = ['in', '"', '\'\'', '″',  'a']
    cm_ids = ['cm', 'mm', 'm']
    
    if any(keyword in s for keyword in inches_ids):
        is_inch = 1
    elif any(keyword in s for keyword in cm_ids):
        is_inch = 0
    elif any(keyword in s for keyword in no_measures_ids):
        is_inch = -1
    else: 
        is_inch = 1 #if no imformation is given of it's measurements, it is assumed that the measures are in inches
        #is_inch = 2 #in case if a problem is detected and some rows need to be flagged
    
    return is_inch

In [40]:
def extract_numbers(df):
    '''
    Se extrae los primeros 2 valores numéricos que se encuentre y se pasan cm si son pulgadas. Se asumen 3 cosas:
    
    - que no hay obras de 3 dimensiones (esculturas, etc...)
    - si vienen las medidas 2 veces, primero se observan las pulgadas
    - 
    '''
    df['height'] = df.dimensiones.str.extract('([0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?)') #normalmente se extrae el primer match
    temp_series = df.dimensiones.str.replace('[0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?', '',regex=True, n=1)
    df['height'] = pd.to_numeric(df['height'])
    df['width'] = temp_series.str.extract('([0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?)')
    df['width'] = pd.to_numeric(df['width'])
    
    return df
    #que las pase como números

In [12]:
def clean_dimensiones(df):
    '''
    Limpia la columna dimensiones y agrega las columnas de height y width
    '''
    df['dimensiones'] = df.dimensiones.apply(lambda x: fraction_char_replace(x))
    df['dimensiones'] = df.dimensiones.apply(lambda x: fraction_str_replace(x))
    df['is_inch'] = df.dimensiones.apply(lambda x: is_inches(x))
    df = extract_numbers(df)
    
    return df

In [46]:
df_clean = clean_dimensiones(df)

In [42]:
df_clean_2 = extract_numbers(df_clean)